In [1]:
import sys
sys.path.append("../")

from pathlib import Path
from pyosim import Analogs3dOsim
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

In [2]:
DATA_PATH = Path("/home/romain/Downloads/results/mars/1_inverse_kinematic")
PREFIX = 'wu_'

In [3]:
data = pd.DataFrame()
for ifile in DATA_PATH.glob("*.mot"):
    x = Analogs3dOsim().from_mot(ifile).time_normalization(np.linspace(0, 100, 100))
    f = ifile.stem.replace(PREFIX, "")
    d = pd.DataFrame(x.squeeze().T, columns=x.get_2d_labels()).assign(
        filename=f,
        participant=f[:4],
        sex=f[4],
        height=f[-3],
        weight=f[5:].split("H")[0],
        trial=f[-1],
    )
    data = pd.concat([data, d.reset_index()])

In [4]:
height = {
    '1': 'hips-shoulders',
    '2': 'hips-eyes',
    '3': 'shoulders-hips',
    '4': 'shoulders-eyes',
    '5': 'eyes-hips',
    '6': 'eyes-shoulders'
}
data['height'].replace(height, inplace=True)

In [5]:
blacklist = [
    "sternoclavicular_r3",
    "box_rotX",
    "box_rotY",
    "box_rotZ",
    "box_transX",
    "box_transY",
    "box_transZ",
]

data.drop(blacklist, axis=1, inplace=True)

In [6]:
data.head()

,index,thorax_tilt,thorax_list,thorax_rotation,thorax_tx,thorax_ty,thorax_tz,sternoclavicular_r1,sternoclavicular_r2,Acromioclavicular_r1,...,elbow_flexion,pro_sup,hand_r_Add,hand_r_Flex,filename,participant,sex,height,weight,trial
0,0,-16.478680,-5.629521,21.104291,-0.559711,0.512960,0.141451,-8.018814,2.067031,34.158030,...,71.501474,10.471408,-33.034694,9.998702,MarSF12H2_3,MarS,F,hips-eyes,12,3
1,1,-20.071202,-7.276129,21.141765,-0.545550,0.503846,0.134850,-7.252305,2.258725,33.999238,...,72.122015,-1.919988,-37.355677,-2.007850,MarSF12H2_3,MarS,F,hips-eyes,12,3
2,2,-23.071755,-8.453065,21.166014,-0.535030,0.495101,0.129807,-7.208632,2.464077,34.384623,...,75.440166,-1.354588,-33.839406,3.580710,MarSF12H2_3,MarS,F,hips-eyes,12,3
3,3,-25.192844,-9.079397,20.854693,-0.529253,0.488508,0.127389,-6.743033,2.025268,33.017675,...,76.179076,-12.640087,-37.342440,4.937570,MarSF12H2_3,MarS,F,hips-eyes,12,3
4,4,-26.434347,-9.498422,20.650253,-0.528569,0.484715,0.127329,-6.597405,2.679966,33.582012,...,76.170345,-19.133620,-37.787084,4.898930,MarSF12H2_3,MarS,F,hips-eyes,12,3


In [17]:
melted = data.melt(id_vars=['filename', 'participant', 'sex', 'height', 'weight', 'trial', 'index'])
melted.head()

,filename,participant,sex,height,weight,trial,index,variable,value
0,MarSF12H2_3,MarS,F,hips-eyes,12,3,0,thorax_tilt,-16.478680
1,MarSF12H2_3,MarS,F,hips-eyes,12,3,1,thorax_tilt,-20.071202
2,MarSF12H2_3,MarS,F,hips-eyes,12,3,2,thorax_tilt,-23.071755
3,MarSF12H2_3,MarS,F,hips-eyes,12,3,3,thorax_tilt,-25.192844
4,MarSF12H2_3,MarS,F,hips-eyes,12,3,4,thorax_tilt,-26.434347


In [9]:
click = alt.selection_multi(encodings=["color"])

chart = (
    alt.Chart(melted)
    .mark_line()
    .encode(
        x="index",
        y="value",
        detail="filename",
        tooltip="filename",
        color=alt.condition(click, "filename", alt.value("lightgray"), legend=None),
    )
    .properties(selection=click)
    .facet(alt.Row("variable", sort=alt.Sort(x.get_2d_labels())))
    .resolve_scale(y="independent")
)

legend = (
    alt.Chart(melted)
    .mark_point()
    .encode(
        y="filename",
        color=alt.condition(click, "filename", alt.value("lightgray"), legend=None),
    )
).properties(selection=click)

(chart | legend).save('chart.html')

In [ ]:
chart = (
    alt.Chart(melted)
    .mark_line()
    .encode(
        x="index", y="value", detail="filename", tooltip="filename", color="filename"
    )
    .facet(alt.Row("variable", sort=alt.Sort(x.get_2d_labels())), alt.Column("height"))
    .resolve_scale(y="independent")
)

chart

---

In [23]:
melted.shape

(64800, 9)

In [24]:
melted['diff'] = melted.groupby(['filename', 'variable'])['value'].diff()

In [26]:
melted.groupby(['filename', 'variable'])['diff'].sum()

filename     variable            
MarSF12H1_1  Acromioclavicular_r1     4.771284
             Acromioclavicular_r2     5.340656
             Acromioclavicular_r3     1.877218
             elbow_flexion           37.238187
             hand_r_Add             -10.189736
             hand_r_Flex            -25.000501
             pro_sup                -32.319607
             shoulder_ele            -4.755083
             shoulder_plane         -27.007893
             shoulder_rotation       43.931716
             sternoclavicular_r1    -16.216340
             sternoclavicular_r2      2.561625
             thorax_list              5.803918
             thorax_rotation          5.281085
             thorax_tilt             21.983202
             thorax_tx               -0.003014
             thorax_ty                0.079254
             thorax_tz                0.007475
MarSF12H1_2  Acromioclavicular_r1    10.461894
             Acromioclavicular_r2     4.516296
             Acromioclavic

In [25]:
melted.head()

,filename,participant,sex,height,weight,trial,index,variable,value,diff
0,MarSF12H2_3,MarS,F,hips-eyes,12,3,0,thorax_tilt,-16.478680,NaN
1,MarSF12H2_3,MarS,F,hips-eyes,12,3,1,thorax_tilt,-20.071202,-3.592523
2,MarSF12H2_3,MarS,F,hips-eyes,12,3,2,thorax_tilt,-23.071755,-3.000553
3,MarSF12H2_3,MarS,F,hips-eyes,12,3,3,thorax_tilt,-25.192844,-2.121089
4,MarSF12H2_3,MarS,F,hips-eyes,12,3,4,thorax_tilt,-26.434347,-1.241504
